In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
import random

In [6]:
DIRECTORY = r"/home/user/Documents/ML/Deep Learning/CNN/Covid/train"
TEST = r"/home/user/Documents/ML/Deep Learning/CNN/Covid/test"

CATEGORIES = ["Covid","Normal","Viral Pneumonia"]

data = []
for category in CATEGORIES:
    folder = os.path.join(DIRECTORY,category)
    label = CATEGORIES.index(category)
#     print(folder)
    for img in os.listdir(folder):
        img = os.path.join(folder,img)
        img_arr = cv2.imread(img)
        img_arr = cv2.resize(img_arr,(100,100))
#         plt.imshow(img_arr)
#         print(img_arr)
        data.append([img_arr,label])
        

In [7]:
def get_label(index):
    if index == 0:
        return "Covid"
    if index == 1:
        return "Normal"
    if index == 2:
        return "Viral Pneumonia"
    
def show_pic(index): 
    plt.imshow(data[index][0])
    plt.xlabel(get_label(data[index][1]))
    plt.show()
    
# show_pic(5)
# show_pic(1)


In [ ]:
random.shuffle(data)
# show_pic(2)

In [8]:
X = []
Y = []

for features, label in data:
    X.append(features)   
    Y.append(label)
    
    
X = np.asarray(X)
Y = np.asarray(Y)

X = X / 255
X.shape[1:]

(100, 100, 3)

In [9]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

#pre=processing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
    )

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.1
)

train_datagen = train_datagen.flow_from_directory(
    DIRECTORY,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'training'
)

test_datagen = test_datagen.flow_from_directory(
    TEST,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

Found 226 images belonging to 3 classes.
Found 6 images belonging to 3 classes.


In [10]:
train_datagen.class_indices


{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}

In [12]:

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

model = Sequential()
model.add(Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2,strides=2))

model.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

# cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

model.add(Flatten())
model.add(Dense(3,activation='softmax'))

In [13]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(train_datagen,epochs=10,validation_data=test_datagen)

Epoch 1/10
4/4 [==============================] - 10s 3s/step - loss: 1.0788 - accuracy: 0.3894 - val_loss: 1.0563 - val_accuracy: 0.3333
Epoch 2/10
4/4 [==============================] - 9s 2s/step - loss: 0.9496 - accuracy: 0.5531 - val_loss: 0.8841 - val_accuracy: 0.6667
Epoch 3/10
4/4 [==============================] - 9s 2s/step - loss: 0.7806 - accuracy: 0.7212 - val_loss: 0.8300 - val_accuracy: 0.3333
Epoch 4/10
4/4 [==============================] - 9s 2s/step - loss: 0.6560 - accuracy: 0.7345 - val_loss: 0.5171 - val_accuracy: 0.8333
Epoch 5/10
4/4 [==============================] - 11s 2s/step - loss: 0.6255 - accuracy: 0.7611 - val_loss: 0.5639 - val_accuracy: 0.6667
Epoch 6/10
4/4 [==============================] - 10s 2s/step - loss: 0.5723 - accuracy: 0.7168 - val_loss: 0.4295 - val_accuracy: 0.8333
Epoch 7/10
4/4 [==============================] - 10s 2s/step - loss: 0.5513 - accuracy: 0.7920 - val_loss: 0.3646 - val_accuracy: 1.0000
Epoch 8/10
4/4 [=====================

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 112, 112, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_1 (Dense)             (None, 3)                

In [15]:
from tensorflow.keras.models import Model
a = np.array(history.history['accuracy']).mean()
# history.history
model.evaluate_generator(generator=test_datagen)

/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  after removing the cwd from sys.path.


[0.3877217769622803, 0.8333333134651184]

In [18]:
from keras.preprocessing import image
path = "/home/user/Documents/ML/Deep Learning/CNN/Covid/net"
# path = "/home/user/ML/Deep Learning/CNN/Covid/test/Normal"
img = path + "/PN.jpeg"
try:
    
    img_pred = tf.keras.utils.load_img(img,target_size=(IMAGE_SIZE,IMAGE_SIZE))
    img_pred = tf.keras.utils.img_to_array(img_pred)
    img_pred = img_pred / 255
    img_pred = np.expand_dims(img_pred, axis=0)
except:
    print("file not found!")

result = model.predict(img_pred)
# plt.imshow(img_pred)
print(result)
# print(get_label(np.argmax(result)))
print(CATEGORIES[np.argmax(result)])

1/1 [==============================] - 0s 17ms/step
[[7.8972767e-04 1.1240075e-01 8.8680953e-01]]
Viral Pneumonia


In [ ]:
# model.evaluate(X,Y)